<a href="https://colab.research.google.com/github/zivp100/Connect4-RL/blob/master/video_FaceMesh_V01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy==1.19.3
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.9 MB 8.4 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.19.3 which is incompatible.
tensorflow 2.8.2+zzzcolab20220527125636 requires numpy>=1.20, but you have numpy 1.19.3 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.9 MB 265 kB/s 


In [ ]:
import cv2
import time
import os

def video_to_frames(input_loc, output_loc):
    """Function to extract frames from input video file
    and save them as separate frames in an output directory.
    Args:
        input_loc: Input video file.
        output_loc: Output directory to save the frames.
    Returns:
        None
    """
    try:
        os.mkdir(output_loc)
    except OSError:
        pass
    # Log the time
    time_start = time.time()
    # Start capturing the feed
    cap = cv2.VideoCapture(input_loc)
    # Find the number of frames
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    print ("Number of frames: ", video_length)
    count = 0
    print ("Converting video..\n")
    # Start converting the video
    while cap.isOpened():
        # Extract the frame
        ret, frame = cap.read()
        if not ret:
            continue
        # Write the results back to output location.
        cv2.imwrite(output_loc + "/%#05d.jpg" % (count+1), frame)
        count = count + 1
        # If there are no more frames left
        if (count > (video_length-1)):
            # Log the time again
            time_end = time.time()
            # Release the feed
            cap.release()
            # Print stats
            print ("Done extracting frames.\n%d frames extracted" % count)
            print ("It took %d seconds forconversion." % (time_end-time_start))
            break

In [ ]:
filename = 'abnormal'
output_dir = filename + '-output/'
os.mkdir(filename + '-frames/')
os.mkdir(output_dir)

input_loc = filename + '.mp4'
output_loc = filename + '-frames/'
video_to_frames(input_loc, output_loc)

Number of frames:  -1
Converting video..



In [ ]:
video = cv2.VideoCapture(input_loc);
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')
if int(major_ver)  < 3 :
        fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
        print ("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))
else :
        fps = video.get(cv2.CAP_PROP_FPS)
        print ("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))
video.release()


Frames per second using video.get(cv2.CAP_PROP_FPS) : 0.0


In [ ]:
#Rotate images as needed
rotated_dir = filename + '-rotated2/'
os.mkdir(rotated_dir)

for name in sorted(os.listdir(output_loc)):
  image = cv2.rotate(cv2.imread(output_loc + '/' + name), cv2.cv2.ROTATE_90_CLOCKWISE)
  cv2.imwrite(rotated_dir + name, image ) 


Find nose reference point

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

# Read images with OpenCV.
#images = {name: cv2.resize(cv2.imread(rotated_dir + name),(512,512)) for name in sorted(os.listdir(rotated_dir))}

# Preview the images.
#for name, image in images.items():
  #print(name)   
  #cv2_imshow(image)

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

# Read images with OpenCV.
#images = {name: cv2.imread("frames2/" + name) for name in sorted(os.listdir("frames2/"))}
images = {name: cv2.imread(rotated_dir + name) for name in sorted(os.listdir(rotated_dir))}

# Preview the images.
#for name, image in images.items():
  #print(name)   
  #cv2_imshow(image)

In [ ]:
len(images)

3

In [ ]:
os.listdir(rotated_dir)

['base.jpg', '9.jpg', '6.jpg']

In [ ]:
!rm -r abnormal-rotated2/.ipynb_checkpoints

In [ ]:
import mediapipe as mp
import numpy as np
import imutils
mp_face_mesh = mp.solutions.face_mesh

In [ ]:
# Prepare DrawingSpec for drawing the face landmarks later.
mp_drawing = mp.solutions.drawing_utils 
drawing_spec = mp_drawing.DrawingSpec(thickness=2, circle_radius=3)

In [ ]:
text_filename2 = filename + "-results.csv"
with open(text_filename2, 'a') as f:
          f.write('R-X' + ',' + 'R-Y' + ',' + 'R-R' + ',' +
                  'L-X' + ',' + 'L-Y' + ',' + 'L-R' + ',' +
                  'N-X' + ',' + 'N-Y' + ',' + 'dot' + '\n')
          
                  

In [28]:
def find_limbus(image, prefix, point ,side):
    #cv2.imwrite(output_dir + '/' + 'before-' + prefix + name , image ) 
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (15,15), 0)
    edged = cv2.Canny(gray, 50, 75)
    edged = cv2.dilate(edged, None, iterations=1)
    edged = cv2.erode(edged, None, iterations=1)
    cv2.imwrite(output_dir + '/' + prefix + name , edged ) 
    
    
    cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
      cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    # TO DO !!!  Take the leftmost / rightmost cntour (based on the image)
    # side = 0 -> left
    # side = 1 -> right
    # side = 2 -> top


    rectangles = []
    for c in cnts:
      x,y,w,h = cv2.boundingRect(c)
      if (w*h > 250): # remove all small rectangles
        rectangles.append([x,y,w,h])
        cv2.rectangle(image, (x,y), (x+w, y+h), (0,255,255) , 1)
      else:
        cv2.rectangle(image, (x,y), (x+w, y+h), (0,255,255) , -1)
      #cv2.putText(image, str(x), (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1,  (0, 255, 0), 1, cv2.LINE_AA)

    cv2.imwrite(output_dir + '/' + 'before1-' + prefix + name , image ) 

  


    by_x = sorted(rectangles, key=lambda x: x[0])  # x values
    #for x, y, w, h in contours:
    #print(f"{x:4} {y:4} {w:4} {h:4}") 
    if (len(rectangles) > 0):
      if (side==0):
       max_x, max_y, max_w, max_h = by_x[-1]
      elif (side==1):
       max_x, max_y, max_w, max_h = by_x[0]
    else:
      max_x, max_y, max_w, max_h = 0,0,0,0

    
    by_y = sorted(rectangles, key=lambda x: x[1] + x[3])  # x values
    if (len(rectangles) > 0):
      if (side==2):
       max_x, max_y, max_w, max_h = by_y[0]
      elif (side==3) :
       max_x, max_y, max_w, max_h = by_y[-1]
   
    cv2.rectangle(image, (max_x,max_y), (max_x+max_w, max_y+max_h), (255,0,0) , 1)
    return max_x, max_y, max_w, max_h

'''
    # take the biggest rectangle
    max_size = 0
    location = 0
    max_len = 0
    max_x, max_w, max_y, max_h = 0, 0, 0, 0
    print(prefix, ' has ', len(cnts), ' cnts')
    for c in cnts:
      x,y,w,h = cv2.boundingRect(c)
      #epsilon = 0.1*cv2.arcLength(c,False)
      #approx = cv2.approxPolyDP(c,epsilon,False)
      #cv2.drawContours(image, [approx], -1, (0, 0, 255), 3)
      print('len=', cv2.arcLength(c,False))
      current_len = cv2.arcLength(c,False)
      if (current_len > max_len):
        max_len = current_len
        location = c
      #cv2.rectangle(image, (x,y), (x+w, y+h), (0,255,255) , 1)
      #if (w*h > max_size):
      #  max_size = w*h
      #  max_x, max_y, max_w, max_h = x, y, w, h

    if (max_len > 0):
      print("Draw Hull")
      #cv2.drawContours(image, [location], -1, (0, 0, 255), 2)
      hull = cv2.convexHull(c, False)
      cv2.drawContours(image, hull, -1, (0, 0, 255), 1)
      cv2.imwrite(output_dir + '/' + 'before1-' + prefix + name , image ) 


      max_x, max_y, max_w, max_h = cv2.boundingRect(location)

    #IF THE DETECTED EDGE IS THE EDGE OF THE IMAGE THEN IGNORE RECOMMENDATION
    #if (max_x + max_w == image.shape[0]):
    #  print("Max reached")
    #  x,y = point
    #else:
    #  x,y = max_x, max_y


    # CAN WE USE THE DIRECTION OF THE COUNTOUER ???

    #print("found ", max_x, max_y, max_w, max_h, image.shape[0], image.shape[1])
    #cv2.rectangle(image, (max_x,max_y), (max_x+max_w, max_y+max_h), (0,255,255) , 1)
    #cv2.imwrite(output_dir + '/' + 'before-' + prefix + name , image ) 
    #cv2.rectangle(edged, (max_x,max_y), (max_x+max_w, max_y+max_h), (0,255,255) , 1)

    print(max_x, max_w, image.shape[0], image.shape[1])
    ## SAME FOR MIN SIDE  

    return max_x, max_y, max_w  
'''

'\n    # take the biggest rectangle\n    max_size = 0\n    location = 0\n    max_len = 0\n    max_x, max_w, max_y, max_h = 0, 0, 0, 0\n    print(prefix, \' has \', len(cnts), \' cnts\')\n    for c in cnts:\n      x,y,w,h = cv2.boundingRect(c)\n      #epsilon = 0.1*cv2.arcLength(c,False)\n      #approx = cv2.approxPolyDP(c,epsilon,False)\n      #cv2.drawContours(image, [approx], -1, (0, 0, 255), 3)\n      print(\'len=\', cv2.arcLength(c,False))\n      current_len = cv2.arcLength(c,False)\n      if (current_len > max_len):\n        max_len = current_len\n        location = c\n      #cv2.rectangle(image, (x,y), (x+w, y+h), (0,255,255) , 1)\n      #if (w*h > max_size):\n      #  max_size = w*h\n      #  max_x, max_y, max_w, max_h = x, y, w, h\n\n    if (max_len > 0):\n      print("Draw Hull")\n      #cv2.drawContours(image, [location], -1, (0, 0, 255), 2)\n      hull = cv2.convexHull(c, False)\n      cv2.drawContours(image, hull, -1, (0, 0, 255), 1)\n      cv2.imwrite(output_dir + \'/\' + 

In [29]:
with mp_face_mesh.FaceMesh( max_num_faces=1, refine_landmarks=True,
    static_image_mode=True,
    min_detection_confidence=0.45) as face_mesh:
  for name, image in images.items():
    # Convert the BGR image to RGB and process it with MediaPipe Face Mesh.
    #image = cv2.rotate(image, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
    print(f'Face landmarks of {name}:')
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # Draw face landmarks of each face.
   
    if not results.multi_face_landmarks:
      continue
    print(name)

    annotated_image = image.copy()
    for face_landmarks in results.multi_face_landmarks:
       print("found eyes")
       height, width, _ = annotated_image.shape
       nose = [int(face_landmarks.landmark[168].x * width), int(face_landmarks.landmark[168].y * height)]
       #cv2.circle(annotated_image, (nose[0], nose[1]), 3, (0, 0, 255), -1)
  

       leftIrisPoints = [474, 475, 476, 477]
       rightIrisPoints = [469, 470, 471, 472]
       #right, top, left, bottom
    
       left_iris = []
       for p in leftIrisPoints:
             point = [int(face_landmarks.landmark[p].x * width), int(face_landmarks.landmark[p].y * height)]
             left_iris.append(point)

       right_iris = []
       for p in rightIrisPoints:
               point = [int(face_landmarks.landmark[p].x * width), int(face_landmarks.landmark[p].y * height)]
               right_iris.append(point)

    leftIris_leftside  = (int( left_iris[2][0]), int( left_iris[2][1]))
    leftIris_righside  = (int( left_iris[0][0]), int( left_iris[0][1]))
    leftIris_top       = (int( left_iris[1][0]), int( left_iris[1][1]))
    leftIris_bottom    = (int( left_iris[3][0]), int( left_iris[3][1]))
    rightIris_leftside = (int(right_iris[2][0]), int(right_iris[2][1]))
    rightIris_righside = (int(right_iris[0][0]), int(right_iris[0][1]))
    rightIris_top      = (int(right_iris[1][0]), int(right_iris[1][1]))
    rightIris_bottom   = (int(right_iris[3][0]), int(right_iris[3][1]))



    # REDUCE HIGHT A BIT TO PREVENT EYELID
    height_margin = 20

    margin = 25
    '''
    # Left Iris Left Limbus 
    leftIris_leftLimbus_image = annotated_image[int(left_iris[1][1])  + height_margin :int(left_iris[3][1]) - height_margin, 
                                                int(left_iris[2][0]) - margin :int(left_iris[2][0]) + margin, 
                                               :]
    x, y, w = find_limbus(leftIris_leftLimbus_image, 'leftIris_leftLimbus=', leftIris_leftside, 0)
    cv2.rectangle(annotated_image, 
                    (x - margin + int(left_iris[2][0]) , height_margin + int(left_iris[1][1])) ,
                    (x - margin + int(left_iris[2][0]) , height_margin + int(left_iris[3][1])) , 
                    (0,255,0) , 2)
    #cv2.imwrite(output_dir + '/' + 'leftIris-leftLimbus-' + name , leftIris_leftLimbus_image )      

    # Left Iris Right Limbus 
    leftIris_rightLimbus_image = annotated_image[int(left_iris[1][1]) + height_margin :int(left_iris[3][1]) - height_margin, 
                                                int(left_iris[0][0]) - margin :int(left_iris[0][0]) + margin, 
                                               :]
    x, y, w = find_limbus(leftIris_rightLimbus_image, 'leftIris_rightLimbus=', leftIris_righside, 1)
    cv2.rectangle(annotated_image, 
                    (x - margin + w + int(left_iris[0][0]) , height_margin + int(left_iris[1][1])) ,
                    (x - margin + w + int(left_iris[0][0]) , height_margin + int(left_iris[3][1])) , 
                    (0,255,0) , 2)
    #cv2.imwrite(output_dir + '/' + 'leftIris-rightLimbus-' + name , leftIris_rightLimbus_image )   

    # Right Iris Left Limbus
    rightIris_leftLimbus_image = annotated_image[int(right_iris[1][1]) + height_margin :int(right_iris[3][1]) - height_margin, 
                                                int(right_iris[2][0]) - margin :int(right_iris[2][0]) + margin, 
                                               :]
    x, y, w = find_limbus(rightIris_leftLimbus_image, 'rightIris_leftLimbus=', rightIris_leftside, 0)
    cv2.rectangle(annotated_image, 
                    (x - margin + int(right_iris[2][0]) , height_margin + int(right_iris[1][1])) ,
                    (x - margin + int(right_iris[2][0]) , height_margin + int(right_iris[3][1])) , 
                    (0,255,0) , 2)
    #cv2.imwrite(output_dir + '/' + 'rightIris-leftLimbus-' + name , rightIris_leftLimbus_image )      

    # Right Iris Right Limbus
    rightIris_rightLimbus_image = annotated_image[int(right_iris[1][1]) + height_margin :int(right_iris[3][1]) - height_margin, 
                                                 int(right_iris[0][0]) - margin :int(right_iris[0][0]) + margin, 
                                               :]
    x, y, w = find_limbus(rightIris_rightLimbus_image, 'rightIris_rightLimbus=', rightIris_righside, 1)
    cv2.rectangle(annotated_image, 
                    (x - margin + w + int(right_iris[0][0]) , height_margin + int(right_iris[1][1])) ,
                    (x - margin + w + int(right_iris[0][0]) , height_margin + int(right_iris[3][1])) , 
                    (0,255,0) , 2)
    #cv2.imwrite(output_dir + '/' + 'rightIris-rightLimbus-' + name , rightIris_rightLimbus_image )      
'''   
    leftIrisCenter = (int((left_iris[0][0] + left_iris[2][0])/2), int((left_iris[1][1] + left_iris[3][1])/2))
    leftIrisRadius =  int(abs(left_iris[0][0] - left_iris[2][0])/2)
    rightIrisCenter = (int((right_iris[0][0] + right_iris[2][0])/2), int((right_iris[1][1] + right_iris[3][1])/2))
    rightIrisRadius = int(abs(right_iris[0][0] - right_iris[2][0])/2)

    #cv2.circle(annotated_image, leftIrisCenter, leftIrisRadius, (0, 255, 0), 1)
    #cv2.circle(annotated_image, rightIrisCenter, rightIrisRadius, (0, 255, 0), 1)
    #cv2.circle(annotated_image, leftIrisCenter, 5, (0, 255, 0), 3)
    #cv2.circle(annotated_image, rightIrisCenter, 5, (0, 255, 0), 3)

    #cv2.circle(annotated_image, leftIris_leftside, 2, (0, 255, 0), 3) # patient left
    #cv2.putText(annotated_image, 'LL', leftIris_leftside, cv2.FONT_HERSHEY_SIMPLEX, 1,  (0, 255, 0), 2, cv2.LINE_AA)
    
    #cv2.circle(annotated_image, leftIris_righside, 2, (0, 255, 0), 3)
    #cv2.putText(annotated_image, 'LR', leftIris_righside, cv2.FONT_HERSHEY_SIMPLEX, 1,  (0, 255, 0), 2, cv2.LINE_AA)

    
    cv2.circle(annotated_image, leftIris_top, 2, (0, 255, 0), 3) # patient left
    cv2.putText(annotated_image, 'LT', leftIris_top, cv2.FONT_HERSHEY_SIMPLEX, 1,  (0, 255, 0), 2, cv2.LINE_AA)
    
    cv2.circle(annotated_image, leftIris_bottom, 2, (0, 255, 0), 3)
    #cv2.putText(annotated_image, 'LB', leftIris_bottom, cv2.FONT_HERSHEY_SIMPLEX, 1,  (0, 255, 0), 2, cv2.LINE_AA)
    


    #cv2.circle(annotated_image, rightIris_leftside, 2, (0, 255, 0), 3) # patient left
    #cv2.putText(annotated_image, 'RL', rightIris_leftside, cv2.FONT_HERSHEY_SIMPLEX, 1,  (0, 255, 0), 2, cv2.LINE_AA)

    #cv2.circle(annotated_image, rightIris_righside, 2, (0, 255, 0), 3)
    cv2.putText(annotated_image, 'RR', rightIris_righside, cv2.FONT_HERSHEY_SIMPLEX, 1,  (0, 255, 0), 2, cv2.LINE_AA)

    
    cv2.circle(annotated_image, rightIris_top, 2, (0, 255, 0), 3) # patient left
    cv2.putText(annotated_image, 'RT', rightIris_top, cv2.FONT_HERSHEY_SIMPLEX, 1,  (0, 255, 0), 2, cv2.LINE_AA)

    cv2.circle(annotated_image, rightIris_bottom, 2, (0, 255, 0), 3)
    #cv2.putText(annotated_image, 'RB', rightIris_bottom, cv2.FONT_HERSHEY_SIMPLEX, 1,  (0, 255, 0), 2, cv2.LINE_AA)
    
    # Top and bottom

    y_margin = 50
    # Left Iris Top Limbus 
    leftIris_topLimbus_image = annotated_image[int(left_iris[1][1]) : int(left_iris[1][1]) + y_margin, 
                                               int(left_iris[2][0]) : int(left_iris[0][0]), 
                                               :]
    x, y, w, h = find_limbus(leftIris_topLimbus_image, 'leftIris_topLimbus=', leftIris_top, 2)
    cv2.rectangle(annotated_image, 
                 (int(left_iris[2][0]) , int(left_iris[1][1]) + y) ,
                 (int(left_iris[0][0]) , int(left_iris[1][1]) + y) , 
                 (0,255,0) , 2)
    

    # top limbus area
    #cv2.rectangle(annotated_image, 
    #                (int(left_iris[2][0]) , int(left_iris[1][1])) ,
    #                (int(left_iris[0][0]) , int(left_iris[1][1]) + y_margin) , 
    #                (0,255,0) , 2)
    
    # left Iris bottom Limbus
    leftIris_bottomLimbus_image = annotated_image[int(left_iris[3][1]) - y_margin : int(left_iris[3][1]), 
                                                  int(left_iris[2][0]) : int(left_iris[0][0]), 
                                                  :]
    x, y, w, h = find_limbus(leftIris_bottomLimbus_image, 'leftIris_bottomLimbus=', leftIris_bottom, 3)
    cv2.rectangle(annotated_image, 
                  (int(left_iris[2][0]) , int(left_iris[3][1]) - y_margin + y + h) ,
                  (int(left_iris[0][0]) , int(left_iris[3][1]) - y_margin + y + h) , 
                  (0,255,0) , 2)
   
    #cv2.rectangle(annotated_image, 
    #                (int(left_iris[2][0]) , int(left_iris[3][1]) - y_margin) ,
    #                (int(left_iris[0][0]) , int(left_iris[3][1])) , 
    #                (0,255,0) , 2)
   
    # Right iris top Limbus
    rightIris_topLimbus_image = annotated_image[int(right_iris[1][1]) : int(right_iris[1][1]) + y_margin, 
                                               int(right_iris[2][0]) : int(right_iris[0][0]), 
                                               :]
    x, y, w, h = find_limbus(rightIris_topLimbus_image, 'rightIris_topLimbus=', rightIris_top, 2)
    cv2.rectangle(annotated_image, 
                 (int(right_iris[2][0]) , int(right_iris[1][1]) + y) ,
                 (int(right_iris[0][0]) , int(right_iris[1][1]) + y) , 
                 (0,255,0) , 2)
    
    # Whole area
    # cv2.rectangle(annotated_image, 
    #                (int(right_iris[2][0]) , int(right_iris[1][1])) ,
    #                (int(right_iris[0][0]) , int(right_iris[1][1]) + y_margin) , 
    #                (0,255,0) , 2)

    # Right iris bottom Limbus
    rightIris_bottomLimbus_image = annotated_image[int(right_iris[3][1]) - y_margin : int(right_iris[3][1]), 
                                                  int(right_iris[2][0]) : int(right_iris[0][0]), 
                                                  :]
    x, y, w, h = find_limbus(rightIris_bottomLimbus_image, 'rightIris_bottomLimbus=', rightIris_bottom, 3)
    cv2.rectangle(annotated_image, 
                  (int(right_iris[2][0]) , int(right_iris[3][1]) - y_margin + y + h) ,
                  (int(right_iris[0][0]) , int(right_iris[3][1]) - y_margin + y + h) , 
                  (0,255,0) , 2)
   
    
    # whole area
    # cv2.rectangle(annotated_image, 
    #                (int(right_iris[2][0]) , int(right_iris[3][1]) - y_margin) ,
    #                (int(right_iris[0][0]) , int(right_iris[3][1])) , 
    #                (0,255,0) , 2)
   





    # Find the sticker
    x_adjustment = 50
    x1 = (int(face_landmarks.landmark[109].x * width), int(face_landmarks.landmark[109].y * height) - x_adjustment)
    x4 = (int(face_landmarks.landmark[336].x * width), int(face_landmarks.landmark[336].y * height))
    #cv2.rectangle(annotated_image, x1,  x4, (0,255,255), 2) # yellow
    # print("point=", x1, x4)
    # NEEDS TO BE HIGHER!

    #cv2.rectangle(annotated_image, x1, x4, (0, 0, 255), 2)# red
    annotated_image_copy = annotated_image[x1[1]:x4[1], x1[0]:x4[0], :]
    #cv2.imwrite(output_dir + '/' + 'copy-' + name , annotated_image_copy )   


    gray = cv2.cvtColor(annotated_image_copy, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (11,11), 0)
    edged = cv2.Canny(gray, 10, 100)
    edged = cv2.dilate(edged, None, iterations=1)
    edged = cv2.erode(edged, None, iterations=1)
    #cv2.imwrite(output_dir + '/' + 'edged-' + name , edged )   

    circle_size = 0
    center = (0,0)
    # find contours in the edge map
    cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
      cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    for c in cnts:
	      # if the contour is not sufficiently large, ignore it
        if cv2.contourArea(c) < 75:
		        continue
        print("found dot")
	      # compute the rotated bounding box of the contour
        orig = annotated_image_copy.copy()
        box = cv2.minAreaRect(c)  # (center, (hgight, weight), angle)
        box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
        box = np.array(box, dtype="int")

        center, radius = cv2.minEnclosingCircle(c)

        circle_size = 2 * radius 
        if (circle_size < 180):
            #print("circle_size < 180")
            continue
        print("circle_size=", circle_size)    
        cv2.putText(annotated_image_copy, str(round(2*radius)), 
                 (int(center[0]), int(center[1])), cv2.FONT_HERSHEY_SIMPLEX, 
                 1,  (0, 255, 0), 2, cv2.LINE_AA)    
        cv2.circle(annotated_image_copy, (int(center[0]), int(center[1])), 5, (0,0,255), 3)
        cv2.circle(annotated_image_copy, (int(center[0]), int(center[1])), int(radius), (0,0,255), 2)
        #cv2.imwrite(output_dir + '/' + 'sticker-' + name , annotated_image_copy )  
        
        actual_center=[0,0]
        actual_center[0] = center[0] + x1[0]
        actual_center[1] = center[1] + x1[1]
        cv2.putText(annotated_image, str(round(2*radius)), 
                 (int(actual_center[0]), int(actual_center[1])), cv2.FONT_HERSHEY_SIMPLEX, 
                 1,  (0, 255, 0), 2, cv2.LINE_AA)    
        cv2.circle(annotated_image, (int(actual_center[0]), int(actual_center[1])), 5, (0,0,255), 3)
        cv2.circle(annotated_image, (int(actual_center[0]), int(actual_center[1])), int(radius), (0,0,255), 2)
 
       
  
    print(str(leftIrisCenter[0])  + ',' + str(leftIrisCenter[1])  + ',' + str(leftIrisRadius) + ',' + 
                  str(rightIrisCenter[0]) + ',' + str(rightIrisCenter[1]) + ',' + str(rightIrisRadius) + ',' + 
                  str(nose[0]) + ',' + str(nose[1]) + ',' + str(circle_size))
    with open(text_filename2, 'a') as f:
          f.write(name + ',' + 
                  str(x - margin + int(left_iris[2][0])) + ',' +
                  str(x - margin + w + int(left_iris[0][0])) + ',' +
                  str(x - margin + int(right_iris[2][0])) + ',' +
                  str(x - margin + w + int(right_iris[0][0])) + ',' +
                  str(int(actual_center[0])) + ',' + str(int(actual_center[1])) + ',' +
                  str(circle_size) + '\n')
          '''
                  str(leftIris_leftside[0])  + ',' + str(leftIris_leftside[1])  + ',' + 
                  str(leftIris_righside[0])  + ',' + str(leftIris_righside[1])  + ',' + 
                  str(rightIris_leftside[0])  + ',' + str(rightIris_leftside[1])  + ',' + 
                  str(rightIris_righside[0])  + ',' + str(rightIris_righside[1])  + ',' + 
                  
          '''
                  #str(nose[0]) + ',' + str(nose[1]) + ',' + 
                
    print("writing file:",output_dir + '/' + name)
    cv2.imwrite(output_dir + '/' + name, annotated_image )   
       


Face landmarks of 12.jpg:
12.jpg
found eyes
found dot
circle_size= 189.66909790039062
found dot
found dot
circle_size= 185.2837677001953
1237,2080,57,656,2062,57,961,2061,185.2837677001953
writing file: abnormal-output//12.jpg
Face landmarks of 6.jpg:
6.jpg
found eyes
found dot
circle_size= 203.7537384033203
1257,2015,54,651,1986,56,952,1932,203.7537384033203
writing file: abnormal-output//6.jpg
Face landmarks of base.jpg:
base.jpg
found eyes
found dot
circle_size= 189.21435546875
1207,2130,55,614,2120,56,917,2089,189.21435546875
writing file: abnormal-output//base.jpg


In [ ]:
! git clone https://github.com/zivp100/EyeCareX.git

In [ ]:
import imutils
import numpy as np

In [ ]:
#put all the results in on dataframe
df1 = pd.read_csv('results2.csv')
df2 = pd.read_csv('results.csv')
df  = pd.merge(df1, df2, on='filename')

df['Left_center_X'] = (df['L-TL-X'] + df['L-BP-X'])/2
df['Left_center_Y'] = (df['L-TL-Y'] + df['L-BP-Y'])/2

df['Left_X_from_nose'] =  abs(df['Nose-X'] + df['Left_center_X'])
df['Left_Y_from_nose'] =  abs(df['Nose-Y'] + df['Left_center_Y'])

df['X_change'] = df['Left_X_from_nose'] - df['Left_X_from_nose'].shift(1)
df['Y_change'] = df['Left_Y_from_nose'] - df['Left_Y_from_nose'].shift(1)

In [ ]:
import pandas as pd

#put all the results in on dataframe
df = pd.read_csv('results-a.csv')
#df2 = pd.read_csv('results.csv')
#df  = pd.merge(df1, df2, on='filename')

df['Left_center_X'] = (df['L-TL-X'] + df['L-BP-X'])/2
df['Left_center_Y'] = (df['L-TL-Y'] + df['L-BP-Y'])/2

df['Left_X_from_nose'] =  abs( df['Left_center_X']) #df['Nose-X']) +
df['Left_Y_from_nose'] =  abs( df['Left_center_Y']) #df['Nose-Y']) +

df['X_change'] = df['Left_X_from_nose'] - df['Left_X_from_nose'].shift(1)
df['Y_change'] = df['Left_Y_from_nose'] - df['Left_Y_from_nose'].shift(1)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df['X_change'])
plt.show

In [ ]:
img=[]

for filename in sorted(os.listdir("frames2/")):
    if filename.endswith(".jpg"): 
          print(filename)
          my_img = cv2.imread("frames2/" + filename)
          img.append(my_img)

height,width,layers=img[1].shape

video = cv2.VideoWriter("output.avi",cv2.VideoWriter_fourcc(*"MJPG"), fps,(width,height))

for j in range(len(img)):
    video.write(img[j])

cv2.destroyAllWindows()
video.release()

In [ ]:
!zip -r /content/file2.zip /content/abnormal-output/

In [ ]:
  '''
    with open(text_filename2, 'a') as f:
          f.write(name + ',' + 
                  str(right_iris[0][0]) + ',' + str(right_iris[0][1]) + ',' +  str(leftIrisRadius) + ',' + 
                  str(right_iris[1][1]) + ',' +
                  str(right_iris[2][0]) + ',' + str(right_iris[2][1]) + ',' +
                  str(right_iris[3][0]) + ',' + str(right_iris[3][1]) + ',' +                                                                                                                   
                  str(left_iris[0][0])  + ',' + str(left_iris[0][1]) + ',' +
                  str(left_iris[1][0])  + ',' + str(left_iris[1][1]) + ',' +
                  str(left_iris[2][0])  + ',' + str(left_iris[2][1]) + ',' +
                  str(left_iris[3][0])  + ',' + str(left_iris[3][1]) + ',' +
                  str(nose[0]) + ',' + str(nose[1]) + '\n')

    for index in range(len(right_iris) - 1):
           print(right_iris[index])
           cv2.line(annotated_image, tuple(right_iris[index]), 
                    tuple(right_iris[index + 1]), (0, 255, 0), 1)
           cv2.circle(annotated_image, tuple(right_iris[index]), 2, (0, 255, 0), -1)
    cv2.circle(annotated_image, tuple(right_iris[len(right_iris) - 1]), 2, (0, 255, 0), -1)
    cv2.line(annotated_image, tuple(right_iris[len(right_iris) - 1]), tuple(right_iris[0]), (0, 255, 0), 1)

    for index in range(len(left_iris) - 1):
           cv2.line(annotated_image, tuple(left_iris[index]), 
                    tuple(left_iris[index + 1]), (0, 255, 0), 1)
           cv2.circle(annotated_image, tuple(left_iris[index]), 2, (0, 255, 0), -1)
    cv2.circle(annotated_image, tuple(left_iris[len(left_iris) - 1]), 2, (0, 255, 0), -1)
    cv2.line(annotated_image, tuple(left_iris[len(left_iris) - 1]), tuple(left_iris[0]), (0, 255, 0), 1)
    '''

In [ ]:
output_dir = 'output2/'


In [ ]:
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import argparse
import imutils
import cv2

In [ ]:
with mp_face_mesh.FaceMesh( max_num_faces=1, refine_landmarks=True,
    static_image_mode=True,
    min_detection_confidence=0.45) as face_mesh:
  for name, image in images.items():
    # Convert the BGR image to RGB and process it with MediaPipe Face Mesh.
    #image = cv2.rotate(image, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # Draw face landmarks of each face.
    print(f'Face landmarks of {name}:')
    if not results.multi_face_landmarks:
      continue
    print(name)

    annotated_image = image.copy()
    for face_landmarks in results.multi_face_landmarks:
       height, width, _ = annotated_image.shape
       x1 = (int(face_landmarks.landmark[103].x * width), int(face_landmarks.landmark[103].y * height))
       x2 = [int(face_landmarks.landmark[333].x * width), int(face_landmarks.landmark[333].y * height)]
       x3 = [int(face_landmarks.landmark[225].x * width), int(face_landmarks.landmark[225].y * height)]
       x4 = (int(face_landmarks.landmark[445].x * width), int(face_landmarks.landmark[445].y * height))
       #cv2.rectangle(annotated_image, x1, x4, (0, 0, 255), 2)
       annotated_image = annotated_image[x1[1]:x4[1], x1[0]:x4[0], :]


       gray = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2GRAY)
       #gray = cv2.GaussianBlur(gray, (11,11), 0)
       edged = cv2.Canny(gray, 50, 100)
       edged = cv2.dilate(edged, None, iterations=1)
       edged = cv2.erode(edged, None, iterations=1)

       # find contours in the edge map
       cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
	        cv2.CHAIN_APPROX_SIMPLE)
       cnts = imutils.grab_contours(cnts)
      # sort the contours from left-to-right and initialize the
      # 'pixels per metric' calibration variable
      #(cnts, _) = contours.sort_contours(cnts)
      #pixelsPerMetric = None


       for c in cnts:
	      # if the contour is not sufficiently large, ignore it
        if cv2.contourArea(c) < 100:
		        continue
	      # compute the rotated bounding box of the contour
        orig = annotated_image.copy()
        box = cv2.minAreaRect(c)
        box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
        box = np.array(box, dtype="int")
        # order the points in the contour such that they appear
        # in top-left, top-right, bottom-right, and bottom-left
        # order, then draw the outline of the rotated bounding
        # box
        box = perspective.order_points(box)
        cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)
        # loop over the original points and draw them
        for (x, y) in box:
          cv2.circle(orig, (int(x), int(y)), 5, (0, 0, 255), -1)
  
    cv2.imwrite(output_dir + '/' + name, orig )   
       
       
'''
       mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          #connections=mp_face_mesh.FACE_CONNECTIONS,
          landmark_drawing_spec=drawing_spec,
          connection_drawing_spec=drawing_spec)
      
    #cv2_imshow(annotated_image)
'''    

Face landmarks of 20220528_174701.jpg:
20220528_174701.jpg
Face landmarks of 20220528_174728.jpg:
20220528_174728.jpg
Face landmarks of 20220528_174733.jpg:
20220528_174733.jpg
Face landmarks of 20220528_174737 (1).jpg:
20220528_174737 (1).jpg
Face landmarks of 20220528_174737.jpg:
20220528_174737.jpg
Face landmarks of 20220528_174740.jpg:
20220528_174740.jpg
Face landmarks of 20220528_174745.jpg:
20220528_174745.jpg
Face landmarks of 20220528_174747.jpg:
20220528_174747.jpg
Face landmarks of 20220528_174757.jpg:
20220528_174757.jpg
Face landmarks of 20220528_174801.jpg:
20220528_174801.jpg
Face landmarks of 20220528_174804.jpg:
20220528_174804.jpg


'\n       mp_drawing.draw_landmarks(\n          image=annotated_image,\n          landmark_list=face_landmarks,\n          #connections=mp_face_mesh.FACE_CONNECTIONS,\n          landmark_drawing_spec=drawing_spec,\n          connection_drawing_spec=drawing_spec)\n      \n    #cv2_imshow(annotated_image)\n'